## Pandas ile Veri Kaynaklarına Bağlanm
* `read_` ile başlayan fonksiyonlarla okuma yapılır.
* `to_` ile başlayan fnonkiyonlarda yazma yapılır.

![alt text](Pandas_read_to.png)

In [52]:
# Yas kolonundan önce boşluk var.
# Veli öncesi bir satır boş geçilmiz.
# Tarih kolonu var.
# ondlaık ayracı , olan bir değer var
# boş geçilmiş alanlar var
# boş geçilmiş alanar için Yok ifadesi kullanışmış.

metin = '''
Ad|Soyad| Yas|Tarih|Kilo
Abdullah|Kise|80|2021-09-24|70,2
Ali|Uçan|55|2021-09-23|

Veli|Kaçan|30|Yok|Yok
'''

In [53]:
import io

metinBuffer = io.StringIO(metin) #buffer (memory stream nesnesi) haline getirir. Yani sanki bir dosya gibi okunabilir
metinBuffer.read() #buffer içeriğini okur. Ancak buffer okunduktan sonra tekrar okunamaz. Tek seferlik okunabilir.

'\nAd|Soyad| Yas|Tarih|Kilo\nAbdullah|Kise|80|2021-09-24|70,2\nAli|Uçan|55|2021-09-23|\n\nVeli|Kaçan|30|Yok|Yok\n'

### Pandas ile CSV Okuma

In [54]:
import numpy as np
import pandas as pd

#### read_csv

In [55]:
pd.read_csv(io.StringIO(metin),
            #delimiter='|',
            sep='|'
            )

,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kise,80,2021-09-24,"70,2"
1,Ali,Uçan,55,2021-09-23,NaN
2,Veli,Kaçan,30,Yok,Yok


In [56]:
df = pd.read_csv(io.StringIO(metin),delimiter='|' #delimiter regex olarak verilebilir. Örneğin \s+ gibi
            #,usecols=['Ad','Soyad',"Yas"] #belli kolonları alabiliyoruz
            #,skipinitialspace=True #kolon adlarında başta boşluk varsa onu atlar hata vermeden kolonu tanır
            #----------

            # ,skiprows= 2 #ilk iki satırı atlar
            # ,skipfooter=1 #son satırı atlar
            # ,engine='python' #python motoru ile çalıştırır. Çünkü bazen hızlı çalışan motorlar hata verebiliyor.

            #---------
            #,skip_blank_lines=False #varsayılan olarka boş satırları atlar. False yaparsak boş satırları da okur.
            
            #
            ,na_values=['Yok'] #Yok ifadesini NaN yapar
            ,decimal=',' #ondalık ayracı , olan bir değer varsa onu . yapar
            #,index_col=["Tarih"]
            ,parse_dates=["Tarih"] #Tarih kolonunu tarih olarak okur
            #---------

            ,dtype={
                    "Ad":str,
                    "Soyad":str,
                    " Yas":np.int8,
                    "Kilo":np.float32,
                    }

            )
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Ad      3 non-null      object        
 1   Soyad   3 non-null      object        
 2    Yas    3 non-null      int8          
 3   Tarih   2 non-null      datetime64[ns]
 4   Kilo    1 non-null      float32       
dtypes: datetime64[ns](1), float32(1), int8(1), object(2)
memory usage: 219.0+ bytes


,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kise,80,2021-09-24,70.199997
1,Ali,Uçan,55,2021-09-23,NaN
2,Veli,Kaçan,30,NaT,NaN


#### to_csv ile csv dosyası oluşturmak

In [57]:
import os
os.makedirs("databases",exist_ok=True)

In [59]:
df.to_csv("./databases/kisiler.csv", index=False) #delimiter varsayılan , olur.

#### Converters ile Veri Temizliği
* Belirtilen kolonda verilen fonksiyona göre okuma yapar.
* Veri temizliği için kullanabiliriz.

In [60]:
pd.read_csv("./databases/kisiler.csv",
            converters={"Soyad":str.upper} #Soyad kolonundaki değerleri büyük harfe çevirir.
            )

,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,KISE,80,2021-09-24,70.2
1,Ali,UÇAN,55,2021-09-23,NaN
2,Veli,KAÇAN,30,NaN,NaN


In [63]:
dict_veritemizligi = {"Uçan":"Uçanlar", "Kaçan":"Kaçanlar"}

pd.read_csv("./databases/kisiler.csv",
            converters={"Soyad":lambda x: dict_veritemizligi.get(x,x)} #keylerin değerlerini alır. Eğer key yoksa kolonun değerini döndürür.
            )

,Ad,Soyad,Yas,Tarih,Kilo
0,Abdullah,Kise,80,2021-09-24,70.2
1,Ali,Uçanlar,55,2021-09-23,NaN
2,Veli,Kaçanlar,30,NaN,NaN


### Excel Okuma İşlemleri

In [68]:
df_dict = pd.read_excel("./databases/mykisiler.xlsx"
                        ,sheet_name=None #kisiler #None ile tüm sayfalar okunur. Veya sayfa adı verilir. Veya sayfa indeksi verilir. verilmezse ilk sayfa okunur.
                ) #excel dosyasını okur

df_dict.keys() #sayfa isimlerini verir #dict olarak döndü.

dict_keys(['kisiler', 'Boş'])

In [69]:
df_dict["kisiler"] #sayfa ismi verilerek sayfa içeriği alınır.

,Id,Ad,Soyad,Yas,Tarih,Kilo
0,1,Abdullah,Kise,15,2020-12-31,70.2
1,2,Ali,Uçan,22,2020-11-11,NaN
2,3,Veli,Kaçan,55,NaT,NaN


### RDBMS's Bağlanmak ve Yazmak (SQL)
* `read_sql_query` DBAPI destekliyor. `pyodbc` vb veya `sqlalchemy` kullanarak bağlanabiliriz. sorgu yazıyoruz
* `read_sql` DBAPI destekliyor. `pyodbc` vb veya `sqlalchemy` kullanarak bağlanabiliriz. Tablo adı veya sorgu yazılabilir.
* `read_sql_table` sadece  `sqlalchemy` destekliyor. Tablo adı yazılır.

NOT:
* Yöntemlerin hepsi `sqlalchemy` destekliyor.
* `sqlalchemy` bir ORM (Object Relational Mapping) kütüphanesidir.
  * ORM kütüphanesi, veritabanı tablolarını nesnelere bağlar.
  * Yazılımcılar programlama dilleriyle veritabanı tablolarını yönetebilirler.

#### DBAPI

In [73]:
import pyodbc

#pyodbc.drivers()

con = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=AdventureWorks2012;Trusted_Connection=yes;")
pd.read_sql_query("SELECT TOP 2 Name, Color, ListPrice FROM Production.Product",con)

C:\Users\Abdullah\AppData\Local\Temp\ipykernel_20204\1945373727.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("SELECT TOP 2 Name, Color, ListPrice FROM Production.Product",con)


,Name,Color,ListPrice
0,Adjustable Race,None,0.0
1,Bearing Ball,None,0.0


#### sqlAlchemy

In [74]:
import sqlalchemy as sqla

In [86]:
#engine yani bağlantı tutan bir nesne oluşturulur.
engine = sqla.create_engine("mssql+pyodbc://localhost/AdventureWorks2012?driver=ODBC+Driver+17+for+SQL+Server")

##### Veritabından Okuma

In [89]:
#engine.table_names(schema="Production") #tablo isimlerini verir.

#
pd.read_sql_query("SELECT TOP 2 Name, Color, ListPrice FROM Production.Product",engine)

#
pd.read_sql("SELECT TOP 2 Name, Color, ListPrice FROM Production.Product",engine)

#
#pd.read_sql_table(schema="Production" ,table_name="Product",con=engine)

,Name,Color,ListPrice
0,Adjustable Race,None,0.0
1,Bearing Ball,None,0.0


#### Veritabanına yazma işlemleri

In [91]:
df_kisiler = pd.read_csv("./databases/kisiler.csv")
df_kisiler.to_sql(name="kisiler",con=engine,if_exists="replace",index=False) #kisiler tablosunu oluşturur. Eğer varsa siler ve tekrar oluşturur. 
#replace yerine append yaparsak tabloya ekler.

-1

### HTML okuyalım

In [101]:
url = "https://en.wikipedia.org/wiki/List_of_Lost_episodes"

lst_table = pd.read_html(url) #html sayfasındaki tabloları okur. Liste olarak döner.

#
lst_table.__len__() #12 tablo mevcut.
lst_table[0]

#
# for df in lst_table:
#     #df #görünmez
#     #print(df.head(2))
#     display(df.head(2)) #display notebook özelliğidir. df olarak gösterir.

Season Episodes               Originally aired                \
  Season Episodes Episodes.1         First aired    Last aired   
0      1       25         25  September 22, 2004  May 25, 2005   
1      2       24         24  September 21, 2005  May 24, 2006   
2      3       23         23     October 4, 2006  May 23, 2007   
3      4       14         14    January 31, 2008  May 29, 2008   
4      5       17         17    January 21, 2009  May 13, 2009   
5      6       18         18    February 2, 2010  May 23, 2010   

  Avg. viewers (millions)   Rank  
  Avg. viewers (millions)   Rank  
0                   15.69  15[2]  
1                   15.50  15[3]  
2                   17.84  10[4]  
3                   13.40  17[5]  
4                   10.94  28[6]  
5                   10.08  31[7]

#### Parquet olarak kaydedelim

In [103]:
df_lost_sesion_1 = lst_table[1]

df_lost_sesion_1.to_parquet("./databases/lost_sesion_1.parquet") #parquet formatında kaydeder.

### Df görünümüne Style ve Options ile müdehale edelim 

In [130]:
dfUrun = pd.read_sql_query("SELECT TOP 10 Name, Color, ListPrice FROM Production.Product WHERE ListPrice >0",engine)

In [131]:
#options
#dir(pd.options.display)

#
pd.options.display.max_columns = 5 #max 5 kolon gösterir. diğperleirni gizler
pd.options.display.max_rows = 5 #max 5 satır gösterir. diğerlerini gizler None olursa tüm satırları gösterir.

#farklı kullanım
pd.set_option("display.max_columns",None) #tüm kolonları gösterir

#
pd.options.display.float_format = '{:.6f}'.format #ondalık sayıları 2 hane gösterir.

dfUrun

,Name,Color,ListPrice
0,LL Mountain Seat Assembly,None,133.340000
1,ML Mountain Seat Assembly,None,147.140000
...,...,...,...
8,HL Touring Seat Assembly,None,196.920000
9,"HL Road Frame - Black, 58",Black,1431.500000


In [132]:
#style


dir(dfUrun.style)

dfUrun.style.highlight_null(color="red") #null olanları kırmızı yapar
dfUrun.style.highlight_max(axis=0,color="Green", subset="ListPrice") #indexi gizler

,Name,Color,ListPrice
0,LL Mountain Seat Assembly,None,133.340000
1,ML Mountain Seat Assembly,None,147.140000
2,HL Mountain Seat Assembly,None,196.920000
3,LL Road Seat Assembly,None,133.340000
4,ML Road Seat Assembly,None,147.140000
5,HL Road Seat Assembly,None,196.920000
6,LL Touring Seat Assembly,None,133.340000
7,ML Touring Seat Assembly,None,147.140000
8,HL Touring Seat Assembly,None,196.920000
9,"HL Road Frame - Black, 58",Black,1431.500000
